<a href="https://colab.research.google.com/github/waelrash1/predictive_analytics_DT302/blob/main/algorithms_implementation/SVMClassiifer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Support Vector Machine (SVM)**
Algorithm is derived and implemented, let's first start by explaining the key concepts and the mathematical derivations. Then, we'll map those equations to Python code.

### 1. **SVM Problem Formulation**

The **SVM** seeks to find the optimal hyperplane that separates two classes with the maximum margin. The margin is the distance between the decision boundary and the closest data points (support vectors).

#### Objective:
Given a dataset of $n$ samples $(x_i, y_i)$, where $x_i$ is the input feature vector and $y_i \in \{-1, 1\}$ is the label, we want to find a hyperplane:

$
f(x) = w^T x - b
$

that maximizes the margin between the two classes. The hyperplane should satisfy the condition:

$
y_i (w^T x_i - b) \geq 1 \quad \forall i
$

### 2. **Primal Optimization Problem**

The objective of SVM is to minimize the following cost function (with regularization) and enforce that the points are classified correctly with a margin:

$
\min_{w, b} \frac{1}{2} ||w||^2 \quad \text{subject to} \quad y_i (w^T x_i - b) \geq 1, \quad \forall i
$

Here, $ \frac{1}{2} ||w||^2 $ ensures that we are maximizing the margin. However, this is a **constrained optimization problem**.

### 3. **Lagrange Multipliers (Converting Constrained to Unconstrained)**

To handle the constraints $ y_i (w^T x_i - b) \geq 1 $, we introduce **Lagrange multipliers** $ \alpha_i \geq 0 $, and the optimization problem becomes:

$
L(w, b, \alpha) = \frac{1}{2} ||w||^2 - \sum_{i=1}^{n} \alpha_i [y_i (w^T x_i - b) - 1]
$

Where:
- $ L(w, b, \alpha) $ is the Lagrangian.
- $ \alpha_i $ are the Lagrange multipliers corresponding to each constraint.

The **dual problem** (obtained by minimizing with respect to $ w $ and $ b $) becomes:

$
\min_w \max_{\alpha \geq 0} \left( \frac{1}{2} ||w||^2 - \sum_{i=1}^{n} \alpha_i (y_i (w^T x_i - b) - 1) \right)
$

### 4. **Hinge Loss (Soft Margin SVM)**

In practice, it is hard to perfectly separate all the data points, so we introduce a **slack variable** to allow some misclassification. This gives us the **hinge loss function**:

$
L(w, b) = \frac{1}{2} ||w||^2 + C \sum_{i=1}^{n} \max(0, 1 - y_i (w^T x_i - b))
$

Where:
- $ C $ controls the tradeoff between maximizing the margin and minimizing the misclassification error.

### 5. **Gradient Descent Updates**

The SVM problem can be solved using **gradient descent**. We need to compute the gradients of the loss function with respect to the weights $w$ and bias $b$:

1. **Gradient w.r.t. weights $w$**:
   - If $ y_i (w^T x_i - b) \geq 1 $, the point is correctly classified, so the gradient is:

   $
   \frac{\partial L}{\partial w} = \lambda w
   $

   - If $ y_i (w^T x_i - b) < 1 $ (misclassified), the gradient becomes:

   $
   \frac{\partial L}{\partial w} = \lambda w - y_i x_i
   $

2. **Gradient w.r.t. bias $b$**:
   - If $ y_i (w^T x_i - b) \geq 1 $, the point is correctly classified, so:

   $
   \frac{\partial L}{\partial b} = 0
   $

   - If $ y_i (w^T x_i - b) < 1 $, the gradient is:

   $
   \frac{\partial L}{\partial b} = - y_i
   $

### 6. **SVM Implementation from Scratch in Python**

Now let's combine these concepts and build the SVM classifier step by step.

```python
import numpy as np

# SVM Classifier
class SVM:
    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000):
        self.learning_rate = learning_rate  # Step size for gradient descent
        self.lambda_param = lambda_param    # Regularization parameter (C)
        self.n_iters = n_iters              # Number of iterations
        self.w = None                       # Weight vector
        self.b = None                       # Bias term

    # Function to train the SVM model
    def fit(self, X, y):
        n_samples, n_features = X.shape
        
        # Initialize weights and bias to zero
        self.w = np.zeros(n_features)
        self.b = 0
        
        # Convert class labels from {0, 1} to {-1, 1}
        y_ = np.where(y <= 0, -1, 1)
        
        # Gradient descent for n_iters iterations
        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                # Condition for correctly classified points
                condition = y_[idx] * (np.dot(x_i, self.w) - self.b) >= 1
                if condition:
                    # Gradient update for correctly classified points
                    self.w -= self.learning_rate * (2 * self.lambda_param * self.w)
                else:
                    # Gradient update for misclassified points (hinge loss)
                    self.w -= self.learning_rate * (2 * self.lambda_param * self.w - np.dot(x_i, y_[idx]))
                    self.b -= self.learning_rate * y_[idx]

    # Function to make predictions
    def predict(self, X):
        # Predict the class label based on the sign of the decision boundary
        return np.sign(np.dot(X, self.w) - self.b)

# Example usage:
if __name__ == "__main__":
    from sklearn.datasets import make_blobs
    import matplotlib.pyplot as plt

    # Generate synthetic data
    X, y = make_blobs(n_samples=100, n_features=2, centers=2, random_state=123)
    
    # Convert the labels to {-1, 1} for SVM
    y = np.where(y == 0, -1, 1)
    
    # Initialize and train the SVM model
    clf = SVM(learning_rate=0.001, lambda_param=0.01, n_iters=1000)
    clf.fit(X, y)
    
    # Make predictions
    predictions = clf.predict(X)
    
    # Calculate accuracy
    accuracy = np.mean(predictions == y)
    print(f"SVM Classification accuracy: {accuracy * 100:.2f}%")

    # Plot decision boundary
    def plot_decision_boundary(X, y, clf):
        plt.scatter(X[:, 0], X[:, 1], marker='o', c=y, s=25, cmap=plt.cm.coolwarm)
        ax = plt.gca()
        xlim = ax.get_xlim()
        ylim = ax.get_ylim()

        # Create grid to evaluate model
        xx, yy = np.meshgrid(np.linspace(xlim[0], xlim[1], 50), np.linspace(ylim[0], ylim[1], 50))
        xy = np.vstack([xx.ravel(), yy.ravel()]).T
        Z = clf.predict(xy).reshape(xx.shape)

        # Plot decision boundary and margins
        ax.contour(xx, yy, Z, colors='k', levels=[-1, 0, 1], alpha=0.5, linestyles=['--', '-', '--'])
        plt.show()

    plot_decision_boundary(X, y, clf)
```

### Key Parts of the Code:

- **Initialization**: The weights `w` and bias `b` are initialized to zero. The learning rate controls how much we adjust the weights after each step.
  
- **Training (Gradient Descent)**: The SVM is trained using gradient descent. For each sample, we check if it satisfies the condition $ y_i (w^T x_i - b) \geq 1 $. If the point is correctly classified, we only update the weights using the regularization term. Otherwise, we update both weights and bias using the hinge loss.

- **Prediction**: The model makes predictions based on the sign of the dot product between the weights and input features.

- **Visualization**: The decision boundary is plotted to show the separating hyperplane, and the margins are indicated by dashed lines.

### Summary:

1. **Formulation**: We convert the constrained SVM optimization problem into an unconstrained problem using Lagrange multipliers and hinge loss.
2. **Gradient Descent**: We derive gradients for weights and bias and update them iteratively.
3

In [ ]:
import numpy as np

# SVM Classifier
class SVM:
    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000):
        self.learning_rate = learning_rate  # Step size for gradient descent
        self.lambda_param = lambda_param    # Regularization parameter (C)
        self.n_iters = n_iters              # Number of iterations
        self.w = None                       # Weight vector
        self.b = None                       # Bias term

    # Function to train the SVM model
    def fit(self, X, y):
        n_samples, n_features = X.shape

        # Initialize weights and bias to zero
        self.w = np.zeros(n_features)
        self.b = 0

        # Convert class labels from {0, 1} to {-1, 1}
        y_ = np.where(y <= 0, -1, 1)

        # Gradient descent for n_iters iterations
        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                # Condition for correctly classified points
                condition = y_[idx] * (np.dot(x_i, self.w) - self.b) >= 1
                if condition:
                    # Gradient update for correctly classified points
                    self.w -= self.learning_rate * (2 * self.lambda_param * self.w)
                else:
                    # Gradient update for misclassified points (hinge loss)
                    self.w -= self.learning_rate * (2 * self.lambda_param * self.w - np.dot(x_i, y_[idx]))
                    self.b -= self.learning_rate * y_[idx]

    # Function to make predictions
    def predict(self, X):
        # Predict the class label based on the sign of the decision boundary
        return np.sign(np.dot(X, self.w) - self.b)

# Example usage:
if __name__ == "__main__":
    from sklearn.datasets import make_blobs
    import matplotlib.pyplot as plt

    # Generate synthetic data
    X, y = make_blobs(n_samples=100, n_features=2, centers=2, random_state=123)

    # Convert the labels to {-1, 1} for SVM
    y = np.where(y == 0, -1, 1)

    # Initialize and train the SVM model
    clf = SVM(learning_rate=0.001, lambda_param=0.01, n_iters=1000)
    clf.fit(X, y)

    # Make predictions
    predictions = clf.predict(X)

    # Calculate accuracy
    accuracy = np.mean(predictions == y)
    print(f"SVM Classification accuracy: {accuracy * 100:.2f}%")

    # Plot decision boundary
    def plot_decision_boundary(X, y, clf):
        plt.scatter(X[:, 0], X[:, 1], marker='o', c=y, s=25, cmap=plt.cm.coolwarm)
        ax = plt.gca()
        xlim = ax.get_xlim()
        ylim = ax.get_ylim()

        # Create grid to evaluate model
        xx, yy = np.meshgrid(np.linspace(xlim[0], xlim[1], 50), np.linspace(ylim[0], ylim[1], 50))
        xy = np.vstack([xx.ravel(), yy.ravel()]).T
        Z = clf.predict(xy).reshape(xx.shape)

        # Plot decision boundary and margins
        ax.contour(xx, yy, Z, colors='k', levels=[-1, 0, 1], alpha=0.5, linestyles=['--', '-', '--'])
        plt.show()

    plot_decision_boundary(X, y, clf)

Apologies for the cutoff in my previous message.

### 7. **Example on a Dataset**

Let's apply our SVM implementation to a real dataset. We'll use the **Breast Cancer Wisconsin** dataset from `sklearn.datasets`, which is a binary classification problem.

#### **Dataset Description:**

- The dataset contains features computed from a digitized image of a fine needle aspirate (FNA) of a breast mass.
- The target variable is binary: malignant (1) or benign (0).

#### **Steps:**

1. **Load the Dataset.**
2. **Preprocess the Data.**
3. **Split into Training and Testing Sets.**
4. **Train the SVM Model.**
5. **Evaluate the Model.**

#### **Code Implementation:**

```python
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset
data = load_breast_cancer()
X, y = data.data, data.target

# Feature scaling
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Convert labels from {0,1} to {-1,1}
y = np.where(y == 0, -1, 1)

# Split into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize SVM classifier
clf = SVM(learning_rate=0.001, lambda_param=0.01, n_iters=1000)

# Train the classifier
clf.fit(X_train, y_train)

# Make predictions
predictions = clf.predict(X_test)

# Evaluate accuracy
accuracy = np.mean(predictions == y_test)
print(f"SVM Classification accuracy on test set: {accuracy * 100:.2f}%")
```

#### **Explanation:**

1. **Loading the Dataset:**

   We use `load_breast_cancer()` from `sklearn.datasets` to load the Breast Cancer dataset, which includes features and labels.

   ```python
   data = load_breast_cancer()
   X, y = data.data, data.target
   ```

2. **Preprocessing:**

   - **Feature Scaling:**

     We standardize the features using `StandardScaler()` because SVMs are sensitive to the scale of the input features.

     ```python
     scaler = StandardScaler()
     X = scaler.fit_transform(X)
     ```

   - **Label Conversion:**

     Our SVM implementation expects labels in \{-1, 1\}, so we convert the labels accordingly.

     ```python
     y = np.where(y == 0, -1, 1)
     ```

3. **Splitting the Data:**

   We split the dataset into training and testing sets using an 80-20 split.

   ```python
   from sklearn.model_selection import train_test_split
   X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
   ```

4. **Training the SVM Model:**

   We initialize the SVM classifier with hyperparameters:

   - `learning_rate=0.001`
   - `lambda_param=0.01`
   - `n_iters=1000`

   Then, we train the model using the `fit()` method.

   ```python
   clf = SVM(learning_rate=0.001, lambda_param=0.01, n_iters=1000)
   clf.fit(X_train, y_train)
   ```

   - **Gradient Descent Updates:**

     During training, for each iteration and each sample, we check the condition \( y_i (w^T x_i - b) \geq 1 \):

     - **If the condition is true (correctly classified):**

       Update the weights using the regularization term.

       ```python
       self.w -= self.learning_rate * (2 * self.lambda_param * self.w)
       ```

     - **If the condition is false (misclassified):**

       Update the weights and bias using both the hinge loss and regularization term.

       ```python
       self.w -= self.learning_rate * (2 * self.lambda_param * self.w - np.dot(x_i, y_[idx]))
       self.b -= self.learning_rate * y_[idx]
       ```

5. **Making Predictions and Evaluating the Model:**

   We predict the labels for the test set and calculate the accuracy.

   ```python
   predictions = clf.predict(X_test)
   accuracy = np.mean(predictions == y_test)
   print(f"SVM Classification accuracy on test set: {accuracy * 100:.2f}%")
   ```

   - **Prediction Function:**

     The `predict()` method computes the sign of the linear function \( w^T x - b \) for each sample.

     ```python
     def predict(self, X):
         approx = np.dot(X, self.w) - self.b
         return np.sign(approx)
     ```

#### **Results:**

After running the code, you might get an output like:

```
SVM Classification accuracy on test set: 96.49%
```

**Note:** The actual accuracy may vary due to random initialization and data splitting.

### **Visualization (Optional):**

Since the dataset has 30 features, visualizing the decision boundary directly is not feasible. However, we can use **Principal Component Analysis (PCA)** to reduce the dimensionality to 2D for visualization purposes.

```python
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Reduce to 2D using PCA
pca = PCA(n_components=2)
X_projected = pca.fit_transform(X_test)

# Plotting
plt.figure(figsize=(8,6))
plt.scatter(X_projected[:, 0], X_projected[:, 1], c=predictions, cmap='coolwarm', alpha=0.8)
plt.title('SVM Predictions (Projected onto 2D using PCA)')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()
```

**Note:** This visualization is a projection and may not accurately reflect the true decision boundary in the original feature space.

### **Adjusting Hyperparameters:**

You can experiment with different hyperparameters to see how they affect the model's performance:

- **Learning Rate (`learning_rate`):** A smaller learning rate may require more iterations but can lead to better convergence.
- **Regularization Parameter (`lambda_param`):** Affects the trade-off between maximizing the margin and minimizing classification error.
- **Number of Iterations (`n_iters`):** More iterations can improve accuracy up to a point but will increase computation time.

### **Summary:**

1. **Problem Formulation:**

   - We started with the SVM optimization problem, converting it from a constrained problem to an unconstrained one using Lagrange multipliers.
   - Introduced the hinge loss function to handle misclassifications.

2. **Derivation of Update Rules:**

   - Derived the gradients with respect to the weights and bias for both correctly classified and misclassified points.
   - Developed update equations for gradient descent based on these gradients.

3. **Implementation:**

   - Implemented the SVM classifier in Python from scratch using NumPy.
   - Applied the classifier to the Breast Cancer dataset and evaluated its performance.

4. **Visualization:**

   - Used PCA to project high-dimensional data into 2D for visualization of predictions.

### **Conclusion:**

We have successfully implemented an SVM classifier from scratch and applied it to a real-world dataset. This exercise demonstrates the practical application of SVMs and provides insight into how they work under the hood.

### **Next Steps:**

- **Kernel Trick:** Extend the implementation to handle non-linear data using kernel functions.
- **Multi-class Classification:** Modify the algorithm to handle multi-class problems (e.g., using one-vs-rest approach).
- **Optimization Techniques:** Implement advanced optimization methods like **SMO (Sequential Minimal Optimization)** for faster convergence.

---

Let me know if you have any questions or need further clarification on any part!

In [ ]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# Load the dataset
data = load_breast_cancer()
X, y = data.data, data.target

# Feature scaling
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Convert labels from {0,1} to {-1,1}
y = np.where(y == 0, -1, 1)

# Split into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize SVM classifier
clf = SVM(learning_rate=0.001, lambda_param=0.01, n_iters=1000)

# Train the classifier
clf.fit(X_train, y_train)

# Make predictions
predictions = clf.predict(X_test)

# Evaluate accuracy
accuracy = np.mean(predictions == y_test)
print(f"SVM Classification accuracy on test set: {accuracy * 100:.2f}%")

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Reduce to 2D using PCA
pca = PCA(n_components=2)
X_projected = pca.fit_transform(X_test)

# Plotting
plt.figure(figsize=(8,6))
plt.scatter(X_projected[:, 0], X_projected[:, 1], c=predictions, cmap='coolwarm', alpha=0.8)
plt.title('SVM Predictions (Projected onto 2D using PCA)')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()

To draw the decision boundary on the PCA-transformed 2D space, we first need to project the original data into 2D using PCA and then plot the SVM decision boundary. The decision boundary is defined by the equation \( w^T x - b = 0 \), where \( w \) and \( b \) are the weights and bias learned by the model.

### Steps:

1. **Reduce Data to 2D Using PCA:**
   We will use PCA to transform the high-dimensional feature space (30 features) into 2D.
   
2. **Plot the Decision Boundary:**
   The decision boundary will be a straight line in the 2D plane because of the linear nature of the SVM. We will compute the values for the line based on the projected weights and bias.

3. **Visualize the Decision Boundary Along with the Data:**

Let's implement this:

### Code:

```python
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

# Reduce the training and test set to 2D using PCA
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# SVM classifier from scratch (trained earlier)

# Function to plot decision boundary
def plot_decision_boundary(X, y, clf, pca, title="SVM Decision Boundary"):
    # Create a mesh to plot the decision boundary
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 500), np.linspace(y_min, y_max, 500))

    # Project the mesh grid back to original space and predict using the classifier
    grid_points = np.c_[xx.ravel(), yy.ravel()]
    grid_points_original_space = pca.inverse_transform(grid_points)
    Z = clf.predict(grid_points_original_space)
    Z = Z.reshape(xx.shape)

    # Plot decision boundary
    plt.contourf(xx, yy, Z, alpha=0.8, cmap='coolwarm')
    
    # Plot the actual data points
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm', edgecolors='k', marker='o', alpha=0.8)

    # Plot support vectors (optional, not implemented in this SVM)
    plt.title(title)
    plt.xlabel('PCA Component 1')
    plt.ylabel('PCA Component 2')
    plt.show()

# Plot decision boundary with PCA-transformed data
plot_decision_boundary(X_test_pca, y_test, clf, pca, title="SVM Decision Boundary on PCA-transformed Data")
```

### Explanation:

1. **PCA Transformation:**
   We first transform both the training and test data into 2D using PCA.
   ```python
   pca = PCA(n_components=2)
   X_train_pca = pca.fit_transform(X_train)
   X_test_pca = pca.transform(X_test)
   ```

2. **Mesh Grid:**
   We create a mesh grid in the 2D PCA space, which will be used to evaluate the decision function at various points and plot the decision boundary.
   ```python
   x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
   y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
   xx, yy = np.meshgrid(np.linspace(x_min, x_max, 500), np.linspace(y_min, y_max, 500))
   ```

3. **Predicting for the Grid:**
   We transform the grid points back into the original feature space and use our SVM classifier to predict the class at each grid point.
   ```python
   grid_points_original_space = pca.inverse_transform(grid_points)
   Z = clf.predict(grid_points_original_space)
   Z = Z.reshape(xx.shape)
   ```

4. **Plotting the Boundary:**
   Finally, we use `plt.contourf()` to plot the decision boundary and the actual data points.
   ```python
   plt.contourf(xx, yy, Z, alpha=0.8, cmap='coolwarm')
   plt.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm', edgecolors='k', marker='o', alpha=0.8)
   ```

### Result:

The plot will show the SVM decision boundary as a straight line in the 2D PCA space, separating the two classes (malignant and benign).

Let me know if you'd like more clarification or improvements on the implementation!

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

# Reduce the training and test set to 2D using PCA
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# SVM classifier from scratch (trained earlier)

# Function to plot decision boundary
def plot_decision_boundary(X, y, clf, pca, title="SVM Decision Boundary"):
    # Create a mesh to plot the decision boundary
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 500), np.linspace(y_min, y_max, 500))

    # Project the mesh grid back to original space and predict using the classifier
    grid_points = np.c_[xx.ravel(), yy.ravel()]
    grid_points_original_space = pca.inverse_transform(grid_points)
    Z = clf.predict(grid_points_original_space)
    Z = Z.reshape(xx.shape)

    # Plot decision boundary
    plt.contourf(xx, yy, Z, alpha=0.8, cmap='coolwarm')

    # Plot the actual data points
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm', edgecolors='k', marker='o', alpha=0.8)

    # Plot support vectors (optional, not implemented in this SVM)
    plt.title(title)
    plt.xlabel('PCA Component 1')
    plt.ylabel('PCA Component 2')
    plt.show()

# Plot decision boundary with PCA-transformed data
plot_decision_boundary(X_test_pca, y_test, clf, pca, title="SVM Decision Boundary on PCA-transformed Data")